In [1]:
import pandas as pd
def getNifty100():

    nList = "nifty100"
    url = "https://archives.nseindia.com/content/indices/ind_" + nList + "list.csv"

    return pd.read_csv(url)


index = getNifty100()

In [7]:
index.Industry.value_counts()

Financial Services                23
Fast Moving Consumer Goods        11
Healthcare                        10
Information Technology             7
Metals & Mining                    7
Automobile and Auto Components     7
Oil Gas & Consumable Fuels         6
Consumer Services                  5
Construction Materials             5
Power                              4
Consumer Durables                  4
Chemicals                          4
Services                           2
Telecommunication                  2
Capital Goods                      1
Construction                       1
Realty                             1
Name: Industry, dtype: int64

In [9]:
index.loc[index.Industry=='Capital Goods',:]

,Company Name,Industry,Symbol,Series,ISIN Code
82,Siemens Ltd.,Capital Goods,SIEMENS,EQ,INE003A01024


In [10]:
import nsepy as npy
import datetime
from dateutil.relativedelta import relativedelta, TH
import re

start_date = datetime.date.today() - relativedelta(years=5)
end_date = datetime.date.today()
ohlc_data= pd.DataFrame()
for symbol in index.Symbol:
    ohlc_data = ohlc_data.append(npy.get_history(
        symbol=re.sub("&", "%26", symbol), start=start_date, end=end_date
    ).reset_index())

In [13]:
import pickle
fl = open(r'C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\ohlc_data.pd','wb')
pickle.dump(ohlc_data,fl)

In [2]:
import pickle
fl = open(r'C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\ohlc_data.pd','rb')
ohlc_data = pickle.load(fl)

In [3]:
ohlc_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117067 entries, 0 to 1238
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Date                117067 non-null  object 
 1   Symbol              117067 non-null  object 
 2   Series              117067 non-null  object 
 3   Prev Close          117067 non-null  float64
 4   Open                117067 non-null  float64
 5   High                117067 non-null  float64
 6   Low                 117067 non-null  float64
 7   Last                117067 non-null  float64
 8   Close               117067 non-null  float64
 9   VWAP                117067 non-null  float64
 10  Volume              117067 non-null  object 
 11  Turnover            117067 non-null  float64
 12  Trades              117067 non-null  object 
 13  Deliverable Volume  117067 non-null  object 
 14  %Deliverble         117067 non-null  float64
dtypes: float64(9), object(6)
memory usag

In [4]:
import datetime
ohlc_data = ohlc_data.set_index('Date')

In [5]:
ohlc_data.sort_index(ascending=False,inplace=True)

In [6]:
import numpy as np
price_change = 0.1
price_change_range = 15
feature_range = 30

for indx,dt in ohlc_data.groupby('Symbol'):
    stckData = dt.copy()
    stckData = stckData.sort_index(ascending=True)
    stckData['CloseShifted'] = stckData.Close.shift(-price_change_range)
    stckData['rangeReturn'] = ((stckData.Close/stckData.CloseShifted) -1)
    stckData['rangeReturnDir'] = stckData['rangeReturn'].apply(lambda x : 'VHPC' if x >= price_change else 'VHLC' if x <= -price_change else 'UN' )
    stckData.dropna(inplace=True)
    dtRangeShft = stckData.shift(feature_range).dropna().index

    for dd in dtRangeShft:
        currIndex = np.where(stckData.index == dd)[0][0]
        prevIndex = currIndex - feature_range
        
        stckData.iloc[prevIndex:currIndex,:][['Open','High','Low','Close','VWAP','Volume','Trades','Deliverable Volume','%Deliverble']]
        break
    break
    

In [7]:
stckData.rangeReturnDir.value_counts()

UN      1065
VHLC      83
VHPC      76
Name: rangeReturnDir, dtype: int64

In [9]:
from tensorflow.keras.datasets import mnist 
(train_images,train_lables),(test_images,test_labels) = mnist.load_data()

11501568/11490434 [==============================] - 1s 0us/step


In [11]:
train_images.shape

(60000, 28, 28)